In [1]:
import requests
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
import matplotlib.ticker as ticker

In [2]:
# We are using cleaned data in order to make the process quicker
df = pd.read_csv(r"C:\Users\Gor088\Downloads\311_Service_Requests_2024_Summer - Cleaned.csv", low_memory=False)

In [ ]:
df.head()

In [ ]:
df.columns

In [5]:
df = df[['Created Date', 'Agency', 'Complaint Type', 'Descriptor',
         'Incident Zip', 'Closed Date', 'Borough',
         'Status', 'Latitude', 'Longitude']]

In [ ]:
df

In [8]:
print(df['Created Date'].isna().sum())
print(df['Agency'].isna().sum())
print(df['Complaint Type'].isna().sum())
print(df['Descriptor'].isna().sum())
print(df['Incident Zip'].isna().sum())
print(df['Closed Date'].isna().sum())
print(df['Borough'].isna().sum())
print(df['Status'].isna().sum())
print(df['Latitude'].isna().sum())
print(df['Longitude'].isna().sum())

0
0
0
0
0
0
0
0
0
0


In [7]:
df = df.dropna(subset=['Closed Date'])
df = df.dropna(subset=['Incident Zip'])
df = df.dropna(subset=['Descriptor'])
df = df.dropna(subset=['Longitude'])

In [ ]:
df

In [ ]:
df['Borough'].unique()

In [11]:
df = df[df['Borough'] != 'Unspecified']

In [12]:
#df['Status'].unique()

In [13]:
#df['Status'].value_counts()

In [14]:
#df = df[df['Status'] == 'Closed']

In [15]:
df = df.reset_index(drop=True)

In [16]:
df['Created Date'] = pd.to_datetime(df['Created Date'])
df['Closed Date'] = pd.to_datetime(df['Closed Date'])
df['Resolution Time'] = (df['Closed Date'] - df['Created Date']).dt.total_seconds() / 60
df['Incident Zip'] = df['Incident Zip'].astype(int)

In [17]:
df

,Created Date,Agency,Complaint Type,Descriptor,Incident Zip,Closed Date,Borough,Status,Latitude,Longitude,Resolution Time
0,2024-08-31 02:57:45,NYPD,Noise - Residential,Banging/Pounding,11694,2024-08-31 03:17:19,QUEENS,Closed,40.585700,-73.820281,19.566667
1,2024-08-31 02:57:27,NYPD,Noise - Commercial,Loud Music/Party,10003,2024-08-31 03:02:05,MANHATTAN,Closed,40.738878,-73.989835,4.633333
2,2024-08-31 02:57:26,NYPD,Noise - Street/Sidewalk,Loud Music/Party,10468,2024-08-31 05:05:51,BRONX,Closed,40.866882,-73.909779,128.416667
3,2024-08-31 02:57:03,NYPD,Noise - Commercial,Loud Music/Party,10003,2024-08-31 03:02:06,MANHATTAN,Closed,40.738878,-73.989835,5.050000
4,2024-08-31 02:56:29,NYPD,Noise - Commercial,Loud Music/Party,11205,2024-08-31 03:29:12,BROOKLYN,Closed,40.694764,-73.954620,32.716667
...,...,...,...,...,...,...,...,...,...,...,...
802618,2024-06-01 02:59:02,DOHMH,Rodent,Rat Sighting,11213,2024-06-01 02:59:02,BROOKLYN,Closed,40.669989,-73.935729,0.000000
802619,2024-06-01 02:58:56,NYPD,Noise - Commercial,Loud Music/Party,10013,2024-06-01 03:02:39,MANHATTAN,Closed,40.719643,-74.002247,3.716667
802620,2024-06-01 02:58:41,NYPD,Noise - Street/Sidewalk,Loud Music/Party,10454,2024-06-01 03:03:05,BRONX,Closed,40.806321,-73.918440,4.400000
802621,2024-06-01 02:58:17,NYPD,Noise - Residential,Loud Music/Party,11225,2024-06-01 03:41:33,BROOKLYN,Closed,40.656713,-73.953107,43.266667


In [ ]:
plt.figure(figsize=(10, 6))
df['Borough'].value_counts().plot(kind='bar', title='Complaints by Borough')
plt.show()

In [ ]:
average_resolution_time_per_agency = df.groupby('Borough')['Resolution Time'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
average_resolution_time_per_agency.plot(kind='bar', title='Average Resolution Time Per Borough')

plt.xlabel('Borough')
plt.ylabel('Average Resolution Time (minutes)')

plt.show()

In [ ]:
# top 10 most common complaint types
top_complaints = df['Complaint Type'].value_counts().head(10)

plt.figure(figsize=(10, 6))
top_complaints.plot(kind='bar', title='Most Common Complaints')

plt.show()

In [ ]:
top_agencie = df['Agency'].value_counts().head(10)

plt.figure(figsize=(10, 6))
top_agencie.plot(kind='bar', title = 'Most Complaints by Agency')

plt.show()

In [ ]:
average_resolution_time_per_agency = df.groupby('Agency')['Resolution Time'].mean().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
average_resolution_time_per_agency.plot(kind='bar', title='Average Resolution Time Per Agency')

plt.xlabel('Agency')
plt.ylabel('Average Resolution Time (minutes)')

plt.show()

In [ ]:
m = folium.Map(location=[40.73, -73.92], zoom_start=8)

# Prepare data for HeatMap (each entry: [latitude, longitude, intensity])
heat_data = [[row['Latitude'], row['Longitude'], 1] for index, row in df.iterrows()]  # Each point is treated with intensity 1

# Add heat map to the folium map
HeatMap(heat_data).add_to(m)

# If you're in a Jupyter notebook, just display the map
m

In [ ]:
top_5_agencies = df['Agency'].value_counts().head(5).index

df_top5 = df[df['Agency'].isin(top_5_agencies)].copy()

df_top5.loc[:, 'Hour'] = pd.to_datetime(df_top5['Created Date']).dt.hour

calls_by_hour_agency = df_top5.groupby(['Hour', 'Agency']).size().reset_index(name='total_calls')

pivot_calls = calls_by_hour_agency.pivot(index='Hour', columns='Agency', values='total_calls')

plt.figure(figsize=(12, 6))
pivot_calls.plot(kind='bar', stacked=True, figsize=(12, 6))
plt.title('Total Calls by Hour and Top 5 Agencies')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Calls')
plt.legend(title='Agency', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
pivot_calls = calls_by_hour_agency.pivot(index='Hour', columns='Agency', values='total_calls').fillna(0)

plt.figure(figsize=(12, 6))
pivot_calls.plot(kind='area', stacked=True, alpha=0.8)

plt.title('Total Calls by Hour and Top 5 Agencies (Stacked Area Plot)')
plt.xlabel('Hour of the Day')
plt.ylabel('Total Calls')
plt.tight_layout()
plt.show()

In [25]:
# df.to_csv('311_Service_Requests_2024_Summer.csv', index=False)